## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate       source  \
0  2025-10-13-13-03-23 +0000          nyt   
1  2025-10-13-13-00-32 +0000          nyt   
2  2025-10-13-13-00-02 +0000  startribune   
3  2025-10-13-12-54-42 +0000         wapo   
4  2025-10-13-12-54-42 +0000         wapo   

                                               title  \
0  Live Updates: Hostages and Palestinian Prisone...   
1                                 Here’s the latest.   
2  Living hostages and Palestinian prisoners are ...   
3  Families in Gaza rejoice, others mourn as they...   
4  Live updates: Trump declares ‘dawn of a new Mi...   

                                                link  
0  https://www.nytimes.com/live/2025/10/12/world/...  
1  https://www.nytimes.com/live/2025/10/12/world/...  
2  https://www.startribune.com/israel-prepares-to...  
3                     https://www.washingtonpost.com  
4  https://www.washingtonpost.com/world/2025/10/1...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
6       trump     38
63      china     19
83     israel     17
2    hostages     15
121     hamas     14

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
172  2025-10-12-22-52-05 +0000  nypost   
4    2025-10-13-12-54-42 +0000    wapo   
125  2025-10-13-05-31-15 +0000  nypost   
194  2025-10-12-20-10-31 +0000  nypost   
9    2025-10-13-12-44-13 +0000     nyt   

                                                 title  \
172  Hillary Clinton heaps rare praise on President...   
4    Live updates: Trump declares ‘dawn of a new Mi...   
125  ‘War is over’ Trump declares — as he arrives i...   
194  Jake Sullivan begrudgingly credits Trump for I...   
9    Asia Markets Fall After Trump Threatens New Ta...   

                                                  link  score  
172  https://nypost.com/2025/10/12/us-news/hillary-...    105  
4    https://www.washingtonpost.com/world/2025/10/1...    100  
125  https://nypost.com/2025/10/13/world-news/war-i...     96  
194  https://nypost.com/2025/10/12/us-news/jake-sul...     85  
9    https://www.nytimes.com/2025/10/12/business/tr...     84

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate   source  \
172    105  2025-10-12-22-52-05 +0000   nypost   
4       62  2025-10-13-12-54-42 +0000     wapo   
32      49  2025-10-13-11-28-00 +0000      wsj   
86      47  2025-10-13-09-06-41 +0000   nypost   
131     44  2025-10-13-04-28-00 +0000      wsj   
116     36  2025-10-13-07-05-15 +0000   nypost   
184     30  2025-10-12-21-42-29 +0000   nypost   
120     29  2025-10-13-06-21-00 +0000      bbc   
241     25  2025-10-12-14-35-27 +0000  latimes   
175     24  2025-10-12-22-17-28 +0000   nypost   

                                                 title  \
172  Hillary Clinton heaps rare praise on President...   
4    Live updates: Trump declares ‘dawn of a new Mi...   
32   Joel Mokyr, Philippe Aghion and Peter Howitt w...   
86   Heartwarming moment shows Israeli hostage spea...   
131  Asia Stock Markets Fall as U.S.-China Trade Te...   
116  Fighting breaks out between Hamas and armed cl...   
184  Zelensky suggests Tomahawk missiles could make...   
120  Boy, 11, shoots farm worker dead in tragic Sou...   
241  A Palestinian American activist was killed in ...   
175  NYC council candidate wants former Rep. Malone...   

                                                  link  
172  https://nypost.com/2025/10/12/us-news/hillary-...  
4    https://www.washingtonpost.com/world/2025/10/1...  
32   https://www.wsj.com/economy/nobel-economics-pr...  
86   https://nypost.com/2025/10/13/world-news/mothe...  
131  https://www.wsj.com/finance/stocks/asia-stock-...  
116  https://nypost.com/2025/10/13/world-news/fight...  
184  https://nypost.com/2025/10/12/us-news/zelensky...  
120  https://www.bbc.com/news/articles/ce86dvr03zqo...  
241  https://www.latimes.com/california/story/2025-...  
175  https://nypost.com/2025/10/12/us-news/why-nyc-...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
